# PyTorch for Computer Vision<br/>Example 2 - Fashion MNIST + VGGNET
## TDC SP 2019 - Track: Machine Learning

**After this notebook, you'll be able to:**
- Use a PyTorch dataset for training a Neural Network
- Use a PyTorch model for training a Neural Network
- Make inference in new images

Let's start ... Importing libraries ...

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from PIL import Image
from collections import OrderedDict

import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import datasets, models, transforms

from tqdm.autonotebook import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

import inspect
import time
import os
import copy

In [ ]:
# check if CUDA is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

## 1. Loading and Preparing Data

- Defining a Transform Pipeline, used in Train and Test Sets
- Download Fashion MNIST into Train and Test Sets
- Create Loader for Train, Val and Test Sets
- Visualize some samples

In [ ]:
# Create a transform pipeline (Compose), resizing image to 224x224, transforming it in Tensor and Normalizing using mean = 0.5 and std = 0.5
# Normalize is applied in each channel (RGB) and in the code below does: image = (image - mean) / std, normalizing the image in the range [-1,1].
transform = transforms.Compose([transforms.Resize(224),
                                transforms.Grayscale(num_output_channels=3),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [ ]:
# Loader parameters
batch_size = 1024
num_workers = 8

In [ ]:
# Download Fashion MNIST dataset (from torchvision.datasets) into trainset, passing through transform pipeline
trainset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', train = True, transform=transform, download=True)

In [ ]:
# create training and validation split 
split = int(0.8 * len(trainset))
index_list = list(range(len(trainset)))
train_idx, valid_idx = index_list[:split], index_list[split:]

In [ ]:
## create sampler objects using SubsetRandomSampler
tr_sampler = SubsetRandomSampler(train_idx)
val_sampler = SubsetRandomSampler(valid_idx)

In [ ]:
# Load tr_sampler into trainloader, for using in NN training process
trainloader = torch.utils.data.DataLoader(trainset, sampler=tr_sampler, num_workers=num_workers, batch_size=batch_size)

In [ ]:
# Load val_sampler into valloader, for using in NN training process
valloader = torch.utils.data.DataLoader(trainset, sampler=val_sampler, num_workers=num_workers, batch_size=batch_size)

In [ ]:
# Download Fashion MNIST dataset (from torchvision.datasets) into testset, passing through transform pipeline
testset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', train = False, transform=transform, download = True)

In [ ]:
# Load testset into testloader, for using in NN validation process
testloader = torch.utils.data.DataLoader(testset, num_workers=num_workers, batch_size=batch_size)

In [ ]:
# Print Dataset Stats
print('# training images: ', len(tr_sampler))
print('# validation images: ', len(val_sampler))
print('# test images: ', len(testset))
print('Classes: ', trainset.classes)

In [ ]:
# Visualize Some sample data

# Obtaning first batch of training images, through iterator of DataLoader
dataiter = iter(trainloader)
images, labels = dataiter.next()

labels_map = {0 : 'T-Shirt', 1 : 'Trouser', 2 : 'Pullover', 3 : 'Dress', 4 : 'Coat', 5 : 'Sandal', 6 : 'Shirt',
              7 : 'Sneaker', 8 : 'Bag', 9 : 'Ankle Boot'};

fig = plt.figure(figsize=(16,16));
columns = 4;
rows = 5;
for i in range(1, columns*rows +1):
    img_xy = np.random.randint(len(images));
    img = images[img_xy][0]
    fig.add_subplot(rows, columns, i)
    plt.title(str(labels_map[int(labels[img_xy])]))
    plt.axis('off')
    plt.imshow(img)
plt.show()

## 2. "Create" NN using VGG19 model - Transfer Learning

- Import VGG19 from torchvision models
- Freeze feature layers, avoiding training
- Fine tune output layer (FC) to FMNIST Classes
- Define NN hyperparameters (Loss, Optimizer, Learning Rate)

In [ ]:
# Create model using VGG19 from torchvision models
vgg19 = models.vgg19(pretrained=True)

### VGG 19 Architecture

![VGG](VGG19.jpg)

In [ ]:
# Show VGG19 Architecture
print(vgg19)

In [ ]:
# Show input and output
print(vgg19.classifier[0].in_features) 
print(vgg19.classifier[6].out_features)

In [ ]:
# Freeze training for all "features" layers
for param in vgg19.features.parameters():
    param.requires_grad = False

# Input features - Output layer
num_features_fc = vgg19.classifier[6].in_features

# Num classes - Output layer
num_outputs_fc = len(trainset.classes)

# Fine tuning Output layer (FC)
vgg19.fc = nn.Linear(num_features_fc, num_outputs_fc)

In [ ]:
# Show VGG19 Architecture
print(vgg19)

In [ ]:
# Hyperparameters

# Loss = CrossEntropy
criterion = nn.CrossEntropyLoss()

# Optimize only last layer with SGD (Stochastic Gradient Descent)
optimizer_vgg19 = optim.SGD(vgg19.fc.parameters(), lr=0.001, momentum=0.9)

# Decay Learning Rate by a factor of 0.1 every 10 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_vgg19, step_size=10, gamma=0.1)

## 3. Train Model

- Get batch of images from trainloader
- Clear Gradients
- Forward Prop
- Calculate Loss
- Backward Prop (using Autograd)
- Run Optimizer (SGD) with Hyperparameters

In [ ]:
def calculate_metric(metric_fn, true_y, pred_y):
    # multi class problems need to have averaging method
    if "average" in inspect.getfullargspec(metric_fn).args:
        return metric_fn(true_y, pred_y, average="macro")
    else:
        return metric_fn(true_y, pred_y)
    
def print_scores(p, r, f1, a, batch_size):
    # just an utility printing function
    for name, scores in zip(("precision", "recall", "F1", "accuracy"), (p, r, f1, a)):
        print(f"\t{name.rjust(14, ' ')}: {sum(scores)/batch_size:.4f}")

In [ ]:
def train_model(model, trainloader, valloader, criterion, optimizer, scheduler, num_epochs=25):
    
    start_ts = time.time()
    
    losses = []
    batches = len(trainloader)
    val_batches = len(valloader)
    
    # loop for every epoch (training + evaluation)
    for epoch in range(num_epochs):
        total_loss = 0

        # progress bar (works in Jupyter notebook too!)
        progress = tqdm(enumerate(trainloader), desc="Loss: ", total=batches)

        # ----------------- TRAINING  -------------------- 
        # set model to training
        model.train()

        for i, data in progress:
            X, y = data[0].to(device), data[1].to(device)

            # training step for single batch
            model.zero_grad()
            outputs = model(X)
            loss = criterion(outputs, y)
            loss.backward()
            scheduler.step()

            # getting training quality data
            current_loss = loss.item()
            total_loss += current_loss

            # updating progress bar
            progress.set_description("Loss: {:.4f}".format(total_loss/(i+1)))

        # releasing unnecessary memory in GPU
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

        # ----------------- VALIDATION  ----------------- 
        val_losses = 0
        precision, recall, f1, accuracy = [], [], [], []

        # set model to evaluating (testing)
        model.eval()
        with torch.no_grad():
            for i, data in enumerate(valloader):
                X, y = data[0].to(device), data[1].to(device)

                outputs = model(X) # this get's the prediction from the network

                val_losses += criterion(outputs, y)

                predicted_classes = torch.max(outputs, 1)[1] # get class from network's prediction

                # calculate P/R/F1/A metrics for batch
                for acc, metric in zip((precision, recall, f1, accuracy), 
                                       (precision_score, recall_score, f1_score, accuracy_score)):
                    acc.append(
                        calculate_metric(metric, y.cpu(), predicted_classes.cpu())
                    )

        print(f"Epoch {epoch+1}/{num_epochs}, training loss: {total_loss/batches}, validation loss: {val_losses/val_batches}")
        print_scores(precision, recall, f1, accuracy, val_batches)
        losses.append(total_loss/batches) # for plotting learning curve
    print(f"Training time: {time.time()-start_ts}s")

In [ ]:
train_model(vgg19, trainloader, valloader, criterion, optimizer_vgg19, exp_lr_scheduler, 5)

In [ ]:
# Save the model checkpoint
torch.save(vgg19.state_dict(), 'model.ckpt')

In [ ]:
def visualize_test(model, testloader):

    # obtain one batch of test images
    dataiter = iter(test_loader)
    images, labels = dataiter.next()
    
    labels_map = {0 : 'T-Shirt', 1 : 'Trouser', 2 : 'Pullover', 3 : 'Dress', 4 : 'Coat', 5 : 'Sandal', 6 : 'Shirt',
        7 : 'Sneaker', 8 : 'Bag', 9 : 'Ankle Boot'};
    
    # move model inputs to cuda, if GPU available
    if train_on_gpu:
        images = images.cuda()

    # get sample outputs
    output = model(images)
    
    # convert output probabilities to predicted class
    _, preds_tensor = torch.max(output, 1)
    preds = np.squeeze(preds_tensor.numpy()) if not train_on_gpu else np.squeeze(preds_tensor.cpu().numpy())
            
    fig = plt.figure(figsize=(16,16));
    columns = 2;
    rows = 4;
    for i in range(1, columns*rows +1):
        img_xy = np.random.randint(len(images));
        img = images[img_xy][0]
        fig.add_subplot(rows, columns, i)
        plt.title(str(labels_map[int(labels[img_xy])])+" ("+labels_map[preds[i-1]]+")")
        plt.axis('off')
        plt.imshow(img)
    plt.show()

In [ ]:
visualize_test(vgg19, testloader)

**Useful links that helped me build this notebook**

- https://www.arunprakash.org/2018/12/cnn-fashion-mnist-dataset-pytorch.html
- https://discuss.pytorch.org/t/understanding-transform-normalize/21730
- https://www.pyimagesearch.com/2019/02/11/fashion-mnist-with-keras-and-deep-learning/
- https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html
- https://www.kaggle.com/carloalbertobarbano/vgg16-transfer-learning-pytorch
- https://medium.com/ml2vec/intro-to-pytorch-with-image-classification-on-a-fashion-clothes-dataset-e589682df0c5
- https://www.jianshu.com/p/34e2ef981f9e
- https://solvemprobler.com/blog/2017/09/29/range-of-convolutional-neural-networks-on-fashion-mnist-dataset/
- https://github.com/udacity/deep-learning-v2-pytorch/blob/master/transfer-learning/Transfer_Learning_Solution.ipynb
- https://medium.com/@josh_2774/deep-learning-with-pytorch-9574e74d17ad
- https://discuss.pytorch.org/t/how-to-modify-the-first-and-last-layer-of-pretrained-network/22597/4
- https://zablo.net/blog/post/using-resnet-for-mnist-in-pytorch-tutorial/
- https://www.kaggle.com/anandad/classify-fashion-mnist-with-vgg16